<a href="https://colab.research.google.com/github/Cheetah-lhp/Privacy-Backdoors/blob/main/Decoding_methods_for_language_generation_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers
!pip install -q Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)

In [ ]:

#greedy search
#encode context
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)
greedy_output = model.generate(**model_inputs, max_new_tokens = 40) #so luong token cua output
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure


In [ ]:
#beam search
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5, #so nhanh se duoc xet de tinh xac suat tong the (overall probability)
    early_stopping=True, # (optional) generation is finished when all beam hypotheses reached the EOS token
    no_repeat_ngram_size=2, #(optional) n-grams penalty makes sure that no n-gram appears twice by manually setting the probability of next words that could create an already seen n-gram to 0.
    num_return_sequences=5 #(optional) choose the number n ì highest scoring beams (num_return_sequences <= num_beams !!)
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea to
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time to take a
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea

In [ ]:
#sampling
from transformers import set_seed
set_seed(42)

sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0, #activate sampling and deactivate top_k by setting top_k sampling to 0
    temperature=0.6, # to make the distribution sharperincreasing the likelihood of high probability words and decreasing the likelihood of low probability words) by lowering the so-called temperature of the softmax.
)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, and I was delighted to have him on my show, so I had a chance to see him. I was very impressed with his body, and I am looking forward to seeing what he has to


In [ ]:
#top-k sampling
set_seed(42)

sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50, #only consider the 50 highest probability words
)
print("Output:\n" + 100 *'-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, which is a little unusual in this part of our family. He's a friendly, calm kind of dog, and I've always wanted to have him around, and I always wanted to go with


In [ ]:
#top-p (nucleus) sampling
set_seed(42)

# set top_k to 50
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=0,
    num_return_sequences=3,
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, a male, to hang out in the zoo," Salinsky said. "Even though I've been with him, we're in the same boat. People have not been able to separate the two
1: I enjoy walking with my cute dog and cooking hot dogs, and my favorite thing about American food is the foods we have all been given, which is why I have called our shops "Cracker Jack". And yes, these are American
2: I enjoy walking with my cute dog," Noro, 25, says. "I think there's lots of things I have learned about parenting since I was a young child, and now that I have a mom that also works there,


In [ ]:
!pip install nbconvert

# fix loi file notebook tren github
!jupyter nbconvert --to notebook --inplace --ClearMetadataPreprocessor.enabled=True your_file.ipynb

[NbConvertApp] WARNING | pattern 'your_file.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    E

In [7]:
# gen theo thu vien co data san chu khong phai random theo prompt LLMs
import json
import random
import uuid
from faker import Faker

fake = Faker("vi_VN")

def generate_record():
    return {
        "full_name": fake.name(),
        "email": fake.email(),
        "phone": fake.phone_number(),
        "address": fake.address().replace("\n", ", "),
        "national_id": str(random.randint(100000000, 999999999)),
        "birthdate": fake.date_of_birth(minimum_age=18, maximum_age=70).isoformat(),
        "passport_number": f"C{random.randint(1000000, 9999999)}",
        "credit_card": {"number": " ".join(str(random.randint(1000, 9999)) for _ in range(4)), "cvv": f"{random.randint(100,999)}"},
        "bank_account": str(random.randint(1000000000, 9999999999)),
        "ip_address": fake.ipv4(),
        "device_id": str(uuid.uuid4()),
        "username": fake.user_name()
    }

# sinh file JSON lines
def generate_dataset(n=10000, output="pii_dataset.jsonl"):
    with open(output, "w", encoding="utf-8") as f:
        for i in range(n):
            record = generate_record()
            f.write(json.dumps(record, ensure_ascii=False) + "\n")
            if i < 10:
              print(record)
generate_dataset(50000)




{'full_name': 'Cô Ánh Lê', 'email': 'maijane@example.org', 'phone': '09 4255135', 'address': '70 Mai Hẻm, JaneThị xã, 292557', 'national_id': '483744090', 'birthdate': '1977-10-30', 'passport_number': 'C9372710', 'credit_card': {'number': '4626 2521 7311 1785', 'cvv': '266'}, 'bank_account': '9819179982', 'ip_address': '98.177.227.177', 'device_id': '5aca1548-ed68-4763-869c-e669476c9fd5', 'username': 'ltran'}
{'full_name': 'Huy Thị Đặng', 'email': 'buijohn@example.net', 'phone': '(05) 7057 3289', 'address': '825 Lê Làng, Thành phố JohnXã, 407508', 'national_id': '554826334', 'birthdate': '1972-09-04', 'passport_number': 'C4489102', 'credit_card': {'number': '2602 4915 8533 4508', 'cvv': '346'}, 'bank_account': '6458809106', 'ip_address': '185.102.167.150', 'device_id': 'c853324f-ddab-4e73-a79b-dff4a159d8de', 'username': 'dnguyen'}
{'full_name': 'Vân Nguyễn', 'email': 'btran@example.org', 'phone': '(02) 8606 7719', 'address': '702 Hoàng Tổ, Quận JaneHuyện, 583428', 'national_id': '64143

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, json, re
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

prompt = (
    "Generate ONE short realistic English text or message. "
    "It should look like a real-world sentence, email, notification, report, or casual message. "
    "The content must be completely fictional and synthetic, not about real people. "
    "The text should naturally include some fake personal or sensitive information, such as:\n"
    "- names\n"
    "- email addresses\n"
    "- phone numbers\n"
    "- IDs, license numbers, IMEI, passport numbers\n"
    "- passwords, PINs, account numbers\n"
    "- dates of birth, age, locations, GPS coordinates\n"
    "- IP addresses (IPv4 or IPv6), user agents\n"
    "- medical or educational references\n"
    "Use any combination of these, but not necessarily all of them.\n"
    "Do not use JSON, bullet points, or lists. "
    "Output only the plain text.\n"
)

def gen_pii_text():
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
    )

    text = tokenizer.decode(output[0], skip_special_tokens=True)

    #loai prompt neu bi duplicate va fix chinh ta
    text = text.replace(prompt, "").strip()
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text

output_file = "pii_dataset.txt"

total = 10
batch_size=2

for i in range(10):
    sample = gen_pii_text()
    print(f"[{i+1}] {sample}\n")

with open(output_file, "w", encoding="utf8") as f:
    for _ in tqdm(range(0, total, batch_size), desc="Generating data"):
        text = gen_pii_text()
        f.write(text + "\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[1] This is the same as text, but a lot more complex. Here is a list of examples of what I suggest: The following list is a complete list of the characters, numbers, and strings that are supposed to be shown on the page. The following is an example of the text, but some of the information is missing: The following list is a complete list of the characters,



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2] This is a list of the following: - name - phone number - email address - IP address - country code - email address - country code - email address - country code - address - phone number - email address - country code - address - name - phone number - email address - country code - email address



KeyboardInterrupt: 